In [1]:
import pandas as pd

In [2]:
def custom_compare(x, y):
    if str(x) != str(y):
        raise RuntimeError(f'Ожидаемое значение: {y}. Фактическое: {x}')

# Предобработка данных

In [15]:
def parse_user_action_solution(line):
    """
    Принимает строку в обозначенном формате, а возвращает словарь.
    Ключами в словаре являются строки 'id', 'action' и 'time',
    а значениями — соответствующие ключам значения из обрабатываемой строки.
    
    Аргументы:
        line: Строка, которую необходимо превратить в словарь.
        
    Возвращаемое значение:
        Словарь в обозначенном формате.
    """
    
    pairs = line.strip('() ').split(', ')
    pairs = [x.split(': ') for x in pairs]
    pairs = {key.strip("'"): val.strip("'") for key, val in pairs}
    pairs['time'] = pd.to_datetime(pairs['time'], format='%Y-%m-%d %H:%M:%S')
    pairs['id'] = int(pairs['id'])
    return pairs

In [16]:
def parse_user_action_test():
    example_1_line = "('id': 7281910, 'action': 'click', 'time': '2023-01-01 12:23:54')"
    example_1_res = {
        'id': 7281910,
        'action': 'click',
        'time': pd.Timestamp(year=2023, month=1, day=1, hour=12, minute=23, second=54),
    }
    
    custom_compare(parse_user_action_solution(example_1_line), example_1_res)
    
    example_2_line = "('id': 1, 'action': 'search', 'time': '1997-12-31 23:23:59')"
    example_2_res = {
        'id': 1,
        'action': 'search',
        'time': pd.Timestamp(year=1997, month=12, day=31, hour=23, minute=23, second=59),
    }
    
    custom_compare(parse_user_action_solution(example_2_line), example_2_res)
    
    print('Все тесты прошли успешно!')

In [17]:
parse_user_action_test()

Все тесты прошли успешно!


# Предобработка данных в pandas

In [3]:
user_session = pd.read_csv('user_session_sample.csv')

In [13]:
user_session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100261 entries, 0 to 100260
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           100261 non-null  int64  
 1   user_id      33890 non-null   float64
 2   ip           100261 non-null  object 
 3   action_time  86339 non-null   object 
 4   action       100261 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.8+ MB


## Задание 1

Подсчитайте, сколько пропущенных значений содержится в каждой из колонок таблицы.

In [4]:
user_session.isna().sum()

id                 0
user_id        66371
ip                 0
action_time    13922
action             0
dtype: int64

## Задание 2

Известно, что значение в колонке «action_time» должно совпадать со значением `action_time`, которое хранится внутри значений характеристики «action».

Воспользуйтесь реализованной ранее функцией `parse_user_action_solution`, чтобы восстановить на основе значений из колонки «action» отсутствующую информацию о моментах времени, когда пользователи совершали соответствующие действия.

В качестве ответа укажите число пользовательских действий, которые были совершены в августе 2023-го года, рассчитанное по таблице с восстановленными данными.

In [7]:
user_session.action_time = user_session.action_time.fillna(user_session.action.apply(lambda x: parse_user_action_solution(x)['time']).dt.date)

In [8]:
user_session.action_time = pd.to_datetime(user_session.action_time)

In [21]:
user_session.isna().sum()

id                 0
user_id        66371
ip                 0
action_time        0
action             0
dtype: int64

In [22]:
user_session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100261 entries, 0 to 100260
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id           100261 non-null  int64         
 1   user_id      33890 non-null   float64       
 2   ip           100261 non-null  object        
 3   action_time  100261 non-null  datetime64[ns]
 4   action       100261 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 3.8+ MB


In [24]:
(user_session.action_time.dt.to_period('M') == '2023-08').sum()

8395

# Исследование данных

## Задание 1

In [12]:
def round_to_2(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.
    
    Аргументы:
        x: Число.
        
    Возвращаемое значение:
        Результат округления числа до 2 знаков после запятой.
    """
    
    return round(x, 2)

In [78]:
def get_sessions(action_times, max_delta=40 * 60):
    """
    Разбивает список моментов времени, когда пользователь проявлял активность,
    на пользовательские сессии. 
    
    Аргументы:
        action_times: Список моментов времени, когда пользователь проявлял активность.
                      Гарантируется, что моменты времени упорядочены в порядке возрастания.
           max_delta: Максимальное значение (в секундах) промежутка между двумя активностями пользователя,
                      при котором они считаются относящимися к одной сессии.
                      
    Возвращаемое значение:
        Список пользовательских сессий. Каждая сессия представляется упорядоченным по возрастанию
        списком моментов времени для действий, которые были совершены в эту сессию.
    """
    
    sessions = []
    cur_session = []
    
    prev_time = None
    
    for time in action_times:
        if prev_time is None or (time - prev_time).total_seconds() > max_delta:
            if len(cur_session) > 0:
                sessions.append(cur_session)
                
            cur_session = [time]
        else:
            cur_session.append(time)
            
        prev_time = time
        
    sessions.append(cur_session)
    
    return sessions

In [79]:
def get_avg_session_time(action_times):
    """
    По списку моментов времени, когда пользователь проявлял активность,
    вычисляет среднюю продолжительность его пользовательской сессии
    с точки зрения времени.
    
    Аргументы:
        action_times: Список моментов времени, когда пользователь проявлял активность.
                      Гарантируется, что моменты времени упорядочены в порядке возрастания.
                      
    Возвращаемое значение:
        Средняя продолжительность пользовательской сессии **в секундах**,
        округлённая до $2$-х знаков после запятой.
    """
    
    session_durations = pd.Series()
    
    for session_times in get_sessions(action_times):
        print((session_times[-1] - session_times[0]).total_seconds())
        session_durations.append(pd.Series((session_times[-1] - session_times[0]).total_seconds()))
        print(*session_durations)
    
    return round_to_2(session_durations.mean())
    
    # pass

In [80]:
def get_avg_session_time_test():
    example_1_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=41, second=21),
        pd.Timestamp(year=2023, month=1, day=13, hour=15, minute=41, second=21),
    ]
    
    example_1_res = 23.77
    
    custom_compare(get_avg_session_time(example_1_action_times), example_1_res)
    
    example_2_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=13, hour=15, minute=41, second=21),
    ]
    
    example_2_res = 0
    
    custom_compare(get_avg_session_time(example_2_action_times), example_2_res)
    
    example_3_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=18, second=24),
        pd.Timestamp(year=2023, month=2, day=21, hour=9, minute=42, second=31),
        pd.Timestamp(year=2023, month=2, day=21, hour=10, minute=21, second=9),
        pd.Timestamp(year=2023, month=2, day=21, hour=10, minute=37, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=17, minute=37, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=18, minute=12, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=18, minute=44, second=53),
        pd.Timestamp(year=2023, month=2, day=27, hour=19, minute=2, second=11),
        pd.Timestamp(year=2023, month=8, day=21, hour=10, minute=37, second=46),
    ]
    
    example_3_res = 46.83
    
    custom_compare(get_avg_session_time(example_3_action_times), example_3_res)
    
    print('Все тесты прошли успешно!')

In [81]:
get_avg_session_time_test()

1426.0

0.0



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


RuntimeError: Ожидаемое значение: 23.77. Фактическое: nan

In [9]:
user_session

,id,user_id,ip,action_time,action
0,619,NaN,244.143.138.0,2021-01-20 18:36:32,"('id': 234676, 'action': 'search', 'time': '20..."
1,919428,30083.0,157.169.233.89,2023-12-10 04:13:50,"('id': 541654, 'action': 'search', 'time': '20..."
2,630539,NaN,160.111.52.224,2023-09-26 04:09:44,"('id': 230556, 'action': 'click', 'time': '202..."
3,963478,NaN,247.14.235.202,2023-12-19 21:02:47,"('id': 838067, 'action': 'click', 'time': '202..."
4,421566,NaN,255.2.67.154,2023-07-20 16:04:14,"('id': 924754, 'action': 'search', 'time': '20..."
...,...,...,...,...,...
100256,787682,NaN,235.213.10.189,2023-11-08 03:26:14,"('id': 611537, 'action': 'click', 'time': '202..."
100257,467742,95044.0,11.197.24.55,2023-08-07 18:09:05,"('id': 105315, 'action': 'search', 'time': '20..."
100258,756486,NaN,25.201.68.226,2023-10-30 11:55:48,"('id': 736403, 'action': 'search', 'time': '20..."
100259,585901,NaN,185.35.134.140,2023-09-14 19:06:45,"('id': 614891, 'action': 'search', 'time': '20..."


In [11]:
user_session.iloc[0].action

"('id': 234676, 'action': 'search', 'time': '2021-01-20 18:36:32')"

## Задание 2

Используя функцию `get_avg_session_time`, вычислите среднюю продолжительность пользовательской сессии для пользователей из таблицы `user_session`, с которой вы работали в рамках предыдущих упражнений. Для этого усредните значения средней продолжительности пользовательских сессий по всем пользователям.

**Обратите внимание**, что среднее можно вычислять как по всем пользователям вообще, так и по тем, у кого среднее время пользовательской сессии больше $0$. В рамках задания вычислите значение по всему множеству пользователей.

Аналогично лекции нужно считать, что пользователя уникальным образом идентифицирует IP-адрес, с которого он совершал действия.

Ответ округлите до $2$-х знаков после запятой.

In [7]:
user_session.head()

,id,user_id,ip,action_time,action
0,619,NaN,244.143.138.0,2021-01-20 18:36:32,"('id': 234676, 'action': 'search', 'time': '20..."
1,919428,30083.0,157.169.233.89,2023-12-10 04:13:50,"('id': 541654, 'action': 'search', 'time': '20..."
2,630539,NaN,160.111.52.224,2023-09-26 04:09:44,"('id': 230556, 'action': 'click', 'time': '202..."
3,963478,NaN,247.14.235.202,2023-12-19 21:02:47,"('id': 838067, 'action': 'click', 'time': '202..."
4,421566,NaN,255.2.67.154,2023-07-20 16:04:14,"('id': 924754, 'action': 'search', 'time': '20..."


## Задание 3

### Вопрос 1

У какой доли пользователей среднее значение пользовательской сессии равно $0$?

Ответ округлите до $2$-х знаков после запятой.

### Вопрос 2

Рассчитайте средннюю продолжительность пользовательской сессии только по пользователям, у которых средняя продолжительность пользовательской сессии больше $0$.

Ответ округлите до $2$-х знаков после запятой.